In [2]:
from pyparsing import Word, Literal, alphas, nums, ZeroOrMore, Optional, OneOrMore, CaselessLiteral

menu = 'parsing'
element = 'flag'
flag = True
_case = 0

if menu == 'parsing' and not flag:
    from itertools import chain # создает итератор, возвращающий значения двух последовательностей, как если бы они были одной
    import re # подключение операций над регулярными выражениями

    # funcs - массив функций.
    def sequence(*funcs): # распаковка итерируемого объекта (массива функций). Т.е. одним вызовом производится множество вызовов для каждого из элементов
        if len(funcs) == 0:
            def result(src):
                yield (), src # ключевое слово для возврата из функции с сохранением текущего значения лок. переменных

            return result

        def result(src):
            for arg1, src in funcs[0](src):
                for others, src in sequence(*funcs[1:])(src):
                    yield (arg1,) + others, src

        return result


    number_regex = re.compile(r"(-?(?:0|[1-9]\d*)(?:\.\d+)?(?:[eE][+-]?\d+)?)\s*(.*)", re.DOTALL)


    def parse_number(src):
        match = number_regex.match(src)
        if match is not None:
            number, src = match.groups()
            yield eval(number), src


    string_regex = re.compile(r"('(?:[^\\']|\\['\\/bfnrt]|\\u[0-9a-fA-F]{4})*?')\s*(.*)", re.DOTALL)


    def parse_string(src):
        match = string_regex.match(src)
        if match is not None:
            string, src = match.groups()
            yield eval(string), src


    def parse_word(word, value=None):
        l = len(word)

        def result(src):
            if src.startswith(word):
                yield value, src[l:].lstrip()

        result.__name__ = "parse_%s" % word
        return result


    parse_true = parse_word("true", True)
    parse_false = parse_word("false", False)
    parse_null = parse_word("null", None)


    def parse_value(src):
        for match in chain(
                parse_string(src),
                parse_number(src),
                parse_array(src),
                parse_object(src),
                parse_true(src),
                parse_false(src),
                parse_null(src),
        ):
            yield match
            return


    parse_left_square_bracket = parse_word("[")
    parse_right_square_bracket = parse_word("]")
    parse_empty_array = sequence(parse_left_square_bracket, parse_right_square_bracket)


    def parse_array(src):
        for _, src in parse_empty_array(src):
            yield [], src
            return

        for (_, items, _), src in sequence(
                parse_left_square_bracket,
                parse_comma_separated_values,
                parse_right_square_bracket,
        )(src):
            yield items, src


    parse_comma = parse_word(",")


    def parse_comma_separated_values(src):
        for (value, _, values), src in sequence(
                parse_value,
                parse_comma,
                parse_comma_separated_values
        )(src):
            yield [value] + values, src
            return

        for value, src in parse_value(src):
            yield [value], src


    parse_left_curly_bracket = parse_word("{")
    parse_right_curly_bracket = parse_word("}")
    parse_empty_object = sequence(parse_left_curly_bracket, parse_right_curly_bracket)


    def parse_object(src):
        for _, src in parse_empty_object(src):
            yield {}, src
            return
        for (_, items, _), src in sequence(
                parse_left_curly_bracket,
                parse_comma_separated_keyvalues,
                parse_right_curly_bracket,
        )(src):
            yield items, src


    parse_colon = parse_word(":")


    def parse_keyvalue(src):
        for (key, _, value), src in sequence(
                parse_string,
                parse_colon,
                parse_value
        )(src):
            yield {key: value}, src


    def parse_comma_separated_keyvalues(src):
        for (keyvalue, _, keyvalues), src in sequence(
                parse_keyvalue,
                parse_comma,
                parse_comma_separated_keyvalues,
        )(src):
            keyvalue.update(keyvalues)
            yield keyvalue, src
            return

        for keyvalue, src in parse_keyvalue(src):
            yield keyvalue, src


    def parse(s):
        s = s.strip()
        match = list(parse_value(s))
        if len(match) != 1:
            raise ValueError("not a valid JSON string")
        result, src = match[0]
        if src.strip():
            raise ValueError("not a valid JSON string")
        return result
elif menu == 'parsing' and _case == 1:
    def _skipIgnorables(self, instring, loc):
        exprsFound = True
        while exprsFound:
            exprsFound = False
            for e in self.ignoreExprs:
                try:
                    while 1:
                        loc, dummy = e._parse(instring, loc)
                        exprsFound = True
                except:
                    pass
        return loc


    def _parseNoCache(self, instring, loc, doActions=True, callPreParse=True):
        TRY, MATCH, FAIL = 0, 1, 2
        debugging = (self.debug)  # and doActions)

        if debugging or self.failAction:
            # ~ print ("Match", self, "at loc", loc, "(%d, %d)" % (lineno(loc, instring), col(loc, instring)))
            if self.debugActions[TRY]:
                self.debugActions[TRY](instring, loc, self)
            try:
                if callPreParse and self.callPreparse:
                    preloc = self.preParse(instring, loc)
                else:
                    preloc = loc
                tokensStart = preloc
                if self.mayIndexError or preloc >= len(instring):
                    loc, tokens = self.parseImpl(instring, preloc, doActions)
                else:
                    loc, tokens = self.parseImpl(instring, preloc, doActions)
            except Exception as err:
                # ~ print ("Exception raised:", err)
                if self.debugActions[FAIL]:
                    self.debugActions[FAIL](instring, tokensStart, self, err)
                if self.failAction:
                    self.failAction(instring, tokensStart, self, err)
                raise
        else:
            if callPreParse and self.callPreparse:
                preloc = self.preParse(instring, loc)
            else:
                preloc = loc
            tokensStart = preloc
            if self.mayIndexError or preloc >= len(instring):
                loc, tokens = self.parseImpl(instring, preloc, doActions)
            else:
                loc, tokens = self.parseImpl(instring, preloc, doActions)

        tokens = self.postParse(instring, loc, tokens)

        retTokens = ''
        if self.parseAction and (doActions or self.callDuringTry):
            if debugging:
                try:
                    for fn in self.parseAction:
                        try:
                            tokens = fn(instring, tokensStart, retTokens)
                        except IndexError as parse_action_exc:
                            exc = 'ParseException("exception raised in parse action")'
                            exc.__cause__ = parse_action_exc
                            raise exc

                        if tokens is not None and tokens is not retTokens:
                            retTokens = "ParseResults"
                except Exception as err:
                    # ~ print "Exception raised in user parse action:", err
                    if self.debugActions[FAIL]:
                        self.debugActions[FAIL](instring, tokensStart, self, err)
                    raise
            else:
                for fn in self.parseAction:
                    try:
                        tokens = fn(instring, tokensStart, retTokens)
                    except IndexError as parse_action_exc:
                        exc = ''
                        exc.__cause__ = parse_action_exc
                        raise exc

                    if tokens is not None and tokens is not retTokens:
                        retTokens = 'ParseResults'
        if debugging:
            # ~ print ("Matched", self, "->", retTokens.asList())
            if self.debugActions[MATCH]:
                self.debugActions[MATCH](instring, tokensStart, loc, self, retTokens)

        return loc, retTokens

if menu == 'parsing' and element == 'flag' and flag == True:

    digit = nums
    letter = alphas
    id_var = Word(alphas) ^ Word(alphas) + Word(nums)
    id = Word(nums) ^ id_var
    program_name = id

    operation = Word('+') ^ Word('-') ^ Word('*') ^ Word('/')
    bool = Word('true') ^ Word('false')
    type = CaselessLiteral('integer') ^ CaselessLiteral('boolean')
    id_list = id_var + ZeroOrMore(Word(',') + id_var)
    variable_list = id_list + ':' + type
    variable_declaration = 'var' + variable_list
    section = variable_declaration
    section_list = OneOrMore(section + ';')
    declarative_part = section_list

    compare_operation = Word('>=') ^ Word('>') ^ Word('<') ^ Word('=') ^ Word('<=')
    expr = id + ZeroOrMore(operation + id)
    condition = expr + compare_operation + expr
    cond_operation = Word('or') ^ Word('and')
    cond = ZeroOrMore('not') + bool ^ ZeroOrMore('not') + condition ^ (
            ZeroOrMore('not') + bool + cond_operation + ZeroOrMore('not') + condition) ^ (
                   ZeroOrMore('not') + bool + cond_operation + ZeroOrMore('not') + bool) ^ (
                   ZeroOrMore('not') + condition + cond_operation + ZeroOrMore('not') + condition) ^ (
                   ZeroOrMore('not') + condition + cond_operation + ZeroOrMore('not') + bool)

    _if = Word('if') + cond + Word('then')
    write = 'write' + '(' + expr + ')' + ';' ^ 'write()' + OneOrMore(Word(';'))
    read = 'read' + '(' + expr + ')' + ';' ^ 'read()' + OneOrMore(Word(';'))

    assignment_operator = id + ":=" + expr + OneOrMore(Word(';'))
    assignment_operator_for = id + ":=" + expr
    _for = 'for' + assignment_operator_for + ZeroOrMore('to') + ZeroOrMore('downto') + expr + 'do'

    _statment_if = read ^ write ^ assignment_operator ^ _if + OneOrMore(
        read ^ write ^ assignment_operator ^ _if + OneOrMore(read ^ write ^ assignment_operator))

    _statment_if_begin = _statment_if ^ _if + 'begin' + OneOrMore(
        read ^ write ^ assignment_operator ^ _statment_if ^ _if + 'begin' + OneOrMore(
            read ^ write ^ assignment_operator ^ _statment_if) + 'end' + OneOrMore(Word(';'))) + 'end' + OneOrMore(
        Word(';'))

    statment_for = _statment_if ^ read ^ write ^ assignment_operator ^ _for + OneOrMore(
        _statment_if ^ read ^ write ^ assignment_operator ^ read ^ write ^ assignment_operator ^ _for + OneOrMore(
            _statment_if ^ read ^ write ^ assignment_operator))

    statment_for_begin = statment_for ^ _for + 'begin' + OneOrMore(
        read ^ write ^ assignment_operator ^ statment_for ^ _for + 'begin' + OneOrMore(
            read ^ write ^ assignment_operator ^ statment_for) + 'end' + OneOrMore(
            Word(';'))) + 'end' + OneOrMore(
        Word(';'))

    statment_if = statment_for_begin ^ read ^ write ^ assignment_operator ^ _if + OneOrMore(statment_for_begin ^
                                                                                            read ^ write ^ assignment_operator ^ _if + OneOrMore(
        statment_for_begin ^ read ^ write ^ assignment_operator))

    statment_if_begin = statment_if ^ _if + 'begin' + OneOrMore(
        read ^ write ^ assignment_operator ^ statment_if ^ _if + 'begin' + OneOrMore(
            read ^ write ^ assignment_operator ^ statment_if) + 'end' + OneOrMore(Word(';'))) + 'end' + OneOrMore(
        Word(';'))

    statment_list = OneOrMore(statment_if) ^ OneOrMore(statment_if_begin) ^ OneOrMore(statment_for) ^ OneOrMore(
        statment_for_begin) ^ Word('begin') + Word('end;') + ZeroOrMore(';')

    executable_part = 'begin' + statment_list + 'end.'

    block = declarative_part + executable_part ^ executable_part
    program = 'program' + program_name + ';' + block

    test_list = [
                """
        program test1;
        
        """,

        """
        
        """,

        """
        program test3;
        var a, i:Integer;
        const  b= 45;
        begin
           for i := 1 then 5 do
             write(b);
    end.
        """,

        """
        program test4;
        var a, i:Integer;
        begin
           for i := 1 to 5 do
             write(b);
        end.

        """,

        """
        program test5;
        var a:Integer;
        begin
           a:=1+2+x;
        end.
        """,

        """
        program test6;
        var b:Integer;
        begin
            begin
            end
        end.
        """,

        """
        program test7;
        var b:Integer;
        begin
            begin
            end
        end.
        asdabshahbsdhb
        """,

        """
        program test8;
        begin
          write();
        end.
        """,

        """
        program test9;
        var a1, b, c:integer;
        var p,q: boolean;
        begin
        
        write();
        read();
        if 7>3 then
            read();
        read();
        write();
    
        end.
        """,

        """
        program test10;
        var a1, b, c:integer;
        var p,q: boolean;
        begin
        
            write();
            read();
            for i:=1 to 10 do
                begin
                read();
                end;
            if 7>3 then
                read();
            read();
            write();
    
        end.
        """,

        """
        program test11;
        var a1, b, c:integer;
        var p,q: boolean;
        begin
            if true then
                write();
            for i:= 1 to 10 do
                read();
        end.
        """,

        """
        program test12;
        var a:integer;
        begin
        for i:= 1 to 10 do
            for j:=1 to 10 do
                read();
        end.
        """,

        """
            Program test13;
            var a:integer;
            begin
            for i:= 1 to 10 do
                for j:=1 to 10 do
                    if 7>3 then
                        read();
            end.
            """,

        """
            program test14;
            var a:integer;
            begin
            for i:= 1 to 10 do
                begin
                for j:=1 to 10 do
                    if 7>3 then
                        read();
                end;
            end.
            """,

        """
            program test15;
            var a:integer;
            begin
            for i:= 1 to 10 do
                begin
                for j:=1 to 10 do
                    begin
                    if 7>3 then
                        read();
                    end;
                end;
            end.
            """,

        """
            program test16;
            var a:integer;
            begin
                a:=5;
            end.
            """,

        """
            program test17
            var a:integer
            begin
                a:=5;
            end.
            """,

    ]
    for test in test_list:
        try:
            parse_result = program.parseString(test)
            test_list = test.split()
            if parse_result[-1] == test_list[-1]:
                print('Тест пройден')
                print('-' * 15)
            else:
                print('Тест провален!')
                print('-' * 15)
        except:
            print('Тест провален!')
            print('-' * 15)
        print(test)
        print('=' * 40)

Тест провален!
---------------

        program test1;
        
        
Тест провален!
---------------

        
        
Тест провален!
---------------

        program test3;
        var a, i:Integer;
        const  b= 45;
        begin
           for i := 1 then 5 do
             write(b);
    end.
        
Тест пройден
---------------

        program test4;
        var a, i:Integer;
        begin
           for i := 1 to 5 do
             write(b);
        end.

        
Тест пройден
---------------

        program test5;
        var a:Integer;
        begin
           a:=1+2+x;
        end.
        
Тест пройден
---------------

        program test6;
        var b:Integer;
        begin
            begin
            end
        end.
        
Тест провален!
---------------

        program test7;
        var b:Integer;
        begin
            begin
            end
        end.
        asdabshahbsdhb
        
Тест пройден
---------------

        program test8;
        begin
 